# Converting Tensorflow model to TF-Lite model

- Tensorflow Lite is a lightweight alternative to Tensorflow that only focuses on inference
- After conversion, test TF-Lite model for inference
- Remove Tensorflow dependency

In [1]:
%autosave 0

Autosave disabled


In [2]:
!python -V

Python 3.9.13


In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

tf.__version__

2022-12-17 06:19:11.794816: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


'2.9.1'

In [4]:
!wget https://bit.ly/3PzCqJ2 -O samosa.jpg

--2022-12-17 06:19:14--  https://bit.ly/3PzCqJ2
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ranga4all1/images-for-inference/raw/main/samosa.jpg [following]
--2022-12-17 06:19:14--  https://github.com/ranga4all1/images-for-inference/raw/main/samosa.jpg
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ranga4all1/images-for-inference/main/samosa.jpg [following]
--2022-12-17 06:19:14--  https://raw.githubusercontent.com/ranga4all1/images-for-inference/main/samosa.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.1

In [5]:
model = keras.models.load_model('food101-model.h5')

2022-12-17 06:19:14.601870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:19:14.609169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:19:14.609742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 06:19:14.610706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [6]:
img = load_img('samosa.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [7]:
preds = model.predict(X)

2022-12-17 06:19:18.004129: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-17 06:19:18.610460: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-17 06:19:18.611354: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-17 06:19:18.611393: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-12-17 06:19:18.612234: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-17 06:19:18.612314: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1/1 [==============================] - 2s 2s/step


In [8]:
preds

array([[-1.581987  , -2.3650863 , -2.5601494 ,  1.1118977 , -3.2538712 ,
        -3.1710904 ,  8.922007  , -0.11740025]], dtype=float32)

In [9]:
classes = ['cup_cakes', 'french_fries', 'hamburger', 'pizza', 'ramen', 'onion_rings', 'samosa', 'waffles']

In [10]:
dict(zip(classes, preds[0]))

{'cup_cakes': -1.581987,
 'french_fries': -2.3650863,
 'hamburger': -2.5601494,
 'pizza': 1.1118977,
 'ramen': -3.2538712,
 'onion_rings': -3.1710904,
 'samosa': 8.922007,
 'waffles': -0.11740025}

## Convert Keras to TF-Lite

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('food101-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpe80a7b7x/assets


INFO:tensorflow:Assets written to: /tmp/tmpe80a7b7x/assets
2022-12-17 06:19:40.879000: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-17 06:19:40.879043: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-17 06:19:40.879864: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpe80a7b7x
2022-12-17 06:19:40.906266: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-12-17 06:19:40.906295: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpe80a7b7x
2022-12-17 06:19:40.993055: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-17 06:19:41.016828: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-12-17 06:19:41.527900: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

In [12]:
!ls -lh

total 410M
-rw-rw-r-- 1 jovyan jovyan 1.5K Dec 17 02:48 convert.py
drwxr-sr-x 3 jovyan jovyan 4.0K Dec 17 04:35 data
-rw-r--r-- 1 jovyan jovyan  83M Dec 17 06:07 food101-model.h5
-rw-rw-r-- 1 jovyan jovyan  83M Dec 17 02:15 food101-model-old.h5
-rw-rw-r-- 1 jovyan jovyan  81M Dec 17 06:19 food101-model.tflite
-rw-rw-r-- 1 jovyan jovyan  960 Dec 16 01:13 lambda_function.py
-rw-rw-r-- 1 jovyan jovyan  80M Dec 17 04:55 model_v1.h5
-rw-rw-r-- 1 jovyan jovyan 5.3M Dec 17 06:15 notebook.ipynb
-rw-rw-r-- 1 jovyan jovyan  73K Dec 16 00:43 onion-rings.jpg
-rw-rw-r-- 1 jovyan jovyan  11K Dec 17 05:19 README.md
-rw-rw-r-- 1 jovyan jovyan  122 Dec 17 05:21 requirements.txt
-rw-rw-r-- 1 jovyan jovyan  34K Dec 17 06:19 samosa.jpg
-rw-rw-r-- 1 jovyan jovyan  271 Dec 16 03:29 test-aws.py
-rw-rw-r-- 1 jovyan jovyan  271 Dec 16 01:46 test-local.py
-rw-rw-r-- 1 jovyan jovyan 1.3K Dec 17 02:40 test-tf-model.py
-rw-rw-r-- 1 jovyan jovyan  32K Dec 17 06:18 tflite-notebook.ipynb
-rw-rw-r-- 1 jovyan jovyan 3.

In [13]:
import tensorflow.lite as tflite

In [14]:
interpreter = tflite.Interpreter(model_path='food101-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [15]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [16]:
classes = ['cup_cakes', 'french_fries', 'hamburger', 'pizza', 'ramen', 'onion_rings', 'samosa', 'waffles']

dict(zip(classes, preds[0]))

{'cup_cakes': -1.5819839,
 'french_fries': -2.3650887,
 'hamburger': -2.5601504,
 'pizza': 1.1118973,
 'ramen': -3.2538755,
 'onion_rings': -3.1710863,
 'samosa': 8.922006,
 'waffles': -0.11740105}

## Removing TF dependency

In [17]:
from PIL import Image

In [18]:
with Image.open('samosa.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

/tmp/ipykernel_25670/1959593832.py:2: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((299, 299), Image.NEAREST)


In [19]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [20]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [21]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [22]:
classes = ['cup_cakes', 'french_fries', 'hamburger', 'pizza', 'ramen', 'onion_rings', 'samosa', 'waffles']

dict(zip(classes, preds[0]))

{'cup_cakes': -1.5819839,
 'french_fries': -2.3650887,
 'hamburger': -2.5601504,
 'pizza': 1.1118973,
 'ramen': -3.2538755,
 'onion_rings': -3.1710863,
 'samosa': 8.922006,
 'waffles': -0.11740105}

## Simpler way of doing it

In [23]:
!pip install keras-image-helper

In [24]:
# !pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime
!pip install tflite_runtime==2.7

In [25]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [26]:
interpreter = tflite.Interpreter(model_path='food101-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [27]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [28]:
url = 'https://bit.ly/3PzCqJ2'
X = preprocessor.from_url(url)

In [29]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [30]:
classes = ['cup_cakes', 'french_fries', 'hamburger', 'pizza', 'ramen', 'onion_rings', 'samosa', 'waffles']

dict(zip(classes, preds[0]))

{'cup_cakes': -1.5819833,
 'french_fries': -2.3650885,
 'hamburger': -2.5601525,
 'pizza': 1.111897,
 'ramen': -3.253876,
 'onion_rings': -3.171088,
 'samosa': 8.922009,
 'waffles': -0.11740293}